In [1]:
using Pkg
Pkg.add("ProgressBars")

   Updating registry at `~/.julia/registries/General`
##############################################################             86.3%############ 100.0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [2]:
using Random, LinearAlgebra, Plots
# bring packages into main namespace
using DataFrames             # Data tables are called "DataFrames"
using StatsPlots             # load plotting packages 
using Statistics             # basic statistical functions
using CSV                    # tools for working with CSV files
using Plots, Random, LinearAlgebra, Statistics, SparseArrays
using ProgressBars
include("proxgrad.jl")
pyplot()

Plots.PyPlotBackend()

In [3]:
dataset = CSV.read("Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets.csv")
dataset = dropmissing(dataset)
dataset[:when] = dataset[:Year] .+ ((dataset[:quarter] .-1) ./ 4)
n = size(dataset,1)
r = .1
cuttoff = Int(round(n*r))
idxs = rand(1:n,n)

# We need to hold out a test dataset to report final results on, that is not used at all for model selection.
# Within k-fold cross validation, train and val datasets will be formed from this main train dataset. 
test_dataset  = dataset[idxs[1:cuttoff],:]
train_dataset = dataset[idxs[cuttoff+1:n],:]

n_train = size(train_dataset,1)
n_test = size(test_dataset,1)

@assert  n_train + n_test == n
@show n_train, n_test
head(dataset)

(n_train, n_test) = (181253, 20139)


,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1
,String,Int64,Int64,Int64,Int64,String
1,Table 1a,2010,1,34614,33195,"Salt Lake City, UT"
2,Table 1a,1998,4,30189,31703,"Colorado Springs, CO"
3,Table 1a,1998,4,30198,30852,"Pittsburgh, PA"
4,Table 1a,2009,3,32211,32575,"Las Vegas, NV"
5,Table 1a,1993,4,30255,30852,"Huntsville, AL"
6,Table 1a,2010,4,33198,32575,"Kansas City, MO"


In [10]:
# Do k-fold cross validation and return the average error_metric on the validation set accross the k folds.
function cross_val(featurizer, loss, regularizer, stepsize, error_metric; k=10, dataset=train_dataset)
    X,y = featurizer(dataset)
    n = size(dataset,1)
    r = Int(round(n / k))
    idxs = rand(1:n,n) # to shuffle the dataset
    error = 0
    for i in (1:k)
        val_idxs = r*(i-1)+1:min(r*i,n)
        tr_low = 1:r*(i-1)
        tr_high = r*i+1:n
        if (i == 1)
            tr_idxs = tr_high
        elseif (i == k)
            tr_idxs = tr_low
        else
            tr_idxs = [tr_low ; tr_high ]
        end
        # @show i, val_idxs
        # @show tr_low, tr_high
        X_tr = X[idxs[tr_idxs],:]
        y_tr = y[idxs[tr_idxs]]
        
        X_val = X[idxs[val_idxs],:]
        y_val = y[idxs[val_idxs]]
        
        w = proxgrad(loss, regularizer, X_tr, y_tr; stepsize=stepsize) 
        ŷ_val = X_val * w
        # @show size(y_val)
        # @show size(ŷ_val)
        error += error_metric(ŷ_val, y_val)
    end
    return error / k
end

function extrapolation_val(featurizer, loss, regularizer, stepsize, error_metric; k=10, dataset=train_dataset)
    X,y = featurizer(dataset)
    n = size(X,1)
    
    tr_idxs = dataset[:when].< 2013.5
    val_idxs = dataset[:when].>= 2013.5
    
    X_tr = X[tr_idxs,:]
    y_tr = y[tr_idxs]
    n_tr = size(X_tr,1)

    X_val = X[val_idxs,:]
    y_val = y[val_idxs]
    n_val = size(X_val,1)
    
    @assert n_tr + n_val == n

    w = proxgrad(loss, regularizer, X_tr, y_tr; stepsize=stepsize) 
    ŷ_val = X_val * w
    # @show size(y_val)
    # @show size(ŷ_val)
    
    return error_metric(ŷ_val, y_val)
end

function train_and_validate(featurizer, loss, regularizer, stepsize, error_metric, mode; k=10, dataset=train_dataset)
    if (mode=="interpolation")
        return cross_val(featurizer, loss, regularizer, stepsize, error_metric; k=k, dataset=dataset)
    else
        return extrapolation_val(featurizer, loss, regularizer, stepsize, error_metric; k=k, dataset=dataset)
    end
end

# For each model in models, do k-fold cross validation and calculate the average error_metric
# on the val set accross the k-folds.
# Each model in model is a tuple of the form (featurizer, loss, regularizer, stepsize),
# where a featurizer is a funciton that takes in a dataset and returns X,y. 
# Returns errors for each model, and the index of the best model
function test_models(models, error_metric,mode;k=10, dataset=train_dataset)
    errors = []
    for model in models
        error = train_and_validate(model...,error_metric,mode;k=k,dataset=dataset)
        errors = [errors; error]
    end
    i = argmin(errors)
    println("The best model is model ",i)
    return errors,i
end

test_models (generic function with 1 method)

# Feature Engineering and Selection

In [11]:
cat_labels = [:carrier_lg, :carrier_low, :airportid_1, :airportid_2]
cat_labels_sub = [:carrier_lg, :carrier_low]

#Sets of all categories in a particular column
cats_sets = [unique(dataset[:, label]) for label in cat_labels]
cats_sets_sub = [unique(dataset[:, label]) for label in cat_labels_sub]

"Computes a onehot vector for every entry in column given a set of categories cats"
function onehot(column, cats=unique(column))
    result = zeros(length(column), length(cats))
    for i = 1:length(column)
        for j =1:length(cats)
            if column[i] === cats[j]
                result[i, j] = 1
            end    
        end
    end
    result
end

onehot

In [12]:
labels_real = [
    :when, 
    :citymarketid_1, 
    :citymarketid_2,
    :airportid_1,
    :airportid_2,
    :nsmiles,
    :passengers,
    :large_ms,
    :fare_lg,
    :lf_ms,
    :fare_low
]
labels_real_sub = [
    :when, 
    :nsmiles,
    :passengers,
    :large_ms,
    :fare_lg,
    :lf_ms,
    :fare_low
]

7-element Array{Symbol,1}:
 :when
 :nsmiles
 :passengers
 :large_ms
 :fare_lg
 :lf_ms
 :fare_low

In [16]:
# Baseline featurizer
function feats_0(dataset)
    X = [dataset[:when] dataset[:airportid_1] dataset[:airportid_2] ones(size(dataset,1))]
    y = dataset[:fare]
    return X,y
end

# Only numeric features (treating airportid and cityid as numeric)
function feats_1(dataset)
    real_vals = convert(Matrix, dataset[labels_real])
    X = hcat(real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

# Numeric and categorical combined (treating airportid and cityid as numerical)
function feats_2(dataset)
    real_vals = convert(Matrix, dataset[labels_real])
    cat_vals = hcat([onehot(dataset[:, cat_labels_sub[i]], cats_sets_sub[i]) for i in 1:size(cat_labels_sub, 1)]...)
    X = hcat(cat_vals, real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

# Numeric and categorical combined (treating airportid and cityid as categorical)
function feats_3(dataset)
    real_vals = convert(Matrix, dataset[labels_real_sub])
    cat_vals = hcat([onehot(dataset[:, cat_labels[i]], cats_sets[i]) for i in 1:size(cat_labels, 1)]...)
    X = hcat(cat_vals, real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

MSE(L1,L2) = sum((L1.-L2).^2) / size(L1,1)

MSE (generic function with 1 method)

In [17]:
models_0 = [
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .1), # best model
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .1),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .1),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .1),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .1),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .1),
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .3),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .3),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .3),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .3),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .3),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .3),
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .5),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .5),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .5),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .5),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .5),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .5),
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .7),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .7),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .7),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .7),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .7),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .7),
    (feats_0, 1/n_train*QuadLoss(), 0.25*QuadReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), 0.75*QuadReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), ZeroReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), 0.25*OneReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), 0.5 *OneReg(), .9), 
    (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .9),
    (feats_0, 1/n_train*QuadLoss(), NonNegConstraint(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.25*QuadReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.5 *QuadReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.75*QuadReg(), .9),
    (feats_0, 1/n_train*L1Loss(), ZeroReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.25*OneReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.5 *OneReg(), .9),
    (feats_0, 1/n_train*L1Loss(), 0.75*OneReg(), .9),
    (feats_0, 1/n_train*L1Loss(), NonNegConstraint(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .9),
    (feats_0, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .9)
]

errors,i = test_models(models_0,MSE,"interpolation";k=5)

The best model is model 7


(Any[6444.345557475726, 6472.98953804663, 6500.131543381863, 6546.008837420464, 6589.096649797127, 6521.768697050956, 6342.636575492222, 6556.784702568247, 6689.757303537891, 6517.943037463074  …  9466.618802896113, 9551.943771500364, 8716.193263185833, 8548.678174658155, 8759.12170319696, 8587.562703478276, 8742.891724347352, 8508.947815870908, 8556.73619447369, 8650.317861116724], 7)

In [18]:
models_1 = [
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .1),	
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .1),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .1),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .1),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .1),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .1),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .1),
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .3),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .3),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .3),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .3),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .3),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .3),
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .5),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .5),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .5),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .5),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .5),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .5),
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .7), # best model
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .7),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .7),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .7),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .7),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .7),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .7),
    (feats_1, 1/n_train*QuadLoss(), 0.25*QuadReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), 0.75*QuadReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), 0.25*OneReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), 0.5 *OneReg(), .9), 
    (feats_1, 1/n_train*QuadLoss(), 0.75*OneReg(), .9),
    (feats_1, 1/n_train*QuadLoss(), NonNegConstraint(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.25*QuadReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.5 *QuadReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.75*QuadReg(), .9),
    (feats_1, 1/n_train*L1Loss(), ZeroReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.25*OneReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.5 *OneReg(), .9),
    (feats_1, 1/n_train*L1Loss(), 0.75*OneReg(), .9),
    (feats_1, 1/n_train*L1Loss(), NonNegConstraint(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .9),
    (feats_1, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .9)
]

errors,i = test_models(models_1,MSE,"interpolation";k=5)

The best model is model 100


(Any[6193.683958637899, 6369.7885543895, 6159.407026116647, 6209.911293493888, 6240.685533265163, 6147.46493182767, 6168.789626172593, 6270.813916253303, 6286.725247162316, 6441.091712111615  …  9160.127692566024, 9116.155808964115, 8106.406427607695, 8189.3557863714395, 8169.358392265513, 8241.017942834744, 8111.754209266165, 8163.578112049412, 8181.089373820905, 8386.858769419941], 100)

In [19]:
models_2 = [
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .1),	
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .1),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .1),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .1),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .1),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .1),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .1),
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .3), # best model
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .3),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .3),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .3),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .3),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .3),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .3),
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .5),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .5),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .5),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .5),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .5),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .5),
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .7),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .7),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .7),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .7),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .7),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .7),
    (feats_2, 1/n_train*QuadLoss(), 0.25*QuadReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), 0.75*QuadReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), ZeroReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), 0.25*OneReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), 0.5 *OneReg(), .9), 
    (feats_2, 1/n_train*QuadLoss(), 0.75*OneReg(), .9),
    (feats_2, 1/n_train*QuadLoss(), NonNegConstraint(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.25*QuadReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.5 *QuadReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.75*QuadReg(), .9),
    (feats_2, 1/n_train*L1Loss(), ZeroReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.25*OneReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.5 *OneReg(), .9),
    (feats_2, 1/n_train*L1Loss(), 0.75*OneReg(), .9),
    (feats_2, 1/n_train*L1Loss(), NonNegConstraint(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .9),
    (feats_2, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .9)
]

errors,i = test_models(models_2,MSE,"interpolation";k=5)

The best model is model 34


(Any[6232.571361570314, 6180.622802841628, 6250.959255304691, 6312.730434380543, 6175.897774342484, 6178.931107594684, 6330.953187424765, 6315.105351982684, 6454.008513374136, 6260.369378240269  …  9118.11611054181, 9122.7258381163, 8214.38655561702, 8267.271991756534, 8173.06342403094, 8119.780874260905, 8041.837101772906, 8183.89210955021, 8212.466614065941, 8253.536745583875], 34)

In [27]:
models_3 = [
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .1),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .1),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .1),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .1),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .3),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .3),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .3),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .3),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .5),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .5),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .5),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .5),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .7), # best model
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7), 
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .7),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .7),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .7),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .7),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), ZeroReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.25*OneReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.5 *OneReg(), .9), 
    (feats_3, 1/n_train*QuadLoss(), 0.75*OneReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), NonNegConstraint(), .9), 
    (feats_3, 1/n_train*L1Loss(), 0.25*QuadReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.5 *QuadReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.75*QuadReg(), .9),
    (feats_3, 1/n_train*L1Loss(), ZeroReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.25*OneReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.5 *OneReg(), .9),
    (feats_3, 1/n_train*L1Loss(), 0.75*OneReg(), .9),
    (feats_3, 1/n_train*L1Loss(), NonNegConstraint(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), ZeroReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.25*OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.5 *OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), 0.75*OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.25), NonNegConstraint(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*QuadReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.25*OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.5 *OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), 0.75*OneReg(), .9),
    (feats_3, 1/n_train*QuantileLoss(quantile=0.75), NonNegConstraint(), .9)
]

errors,i = test_models(models_3,MSE,"interpolation";k=5)

The best model is model 97


(Any[3363.740486962134, 3326.437486748972, 3343.962066608503, 3338.1409987981347, 3356.7913314737843, 3363.6002245983477, 3440.803228988387, 3458.1309509866364, 2942.799637950555, 3034.023614874525  …  4812.541883176984, 4803.926281671811, 3775.7622123979086, 3852.887456411874, 3729.6171373749507, 3807.4800749997507, 3815.8600681355665, 3840.341588544459, 3845.7120404363673, 3880.7676250027994], 97)